# Birthdays probing test

In [1]:
import sys
sys.path.append('/home/jxm3/research/deidentification/unsupervised-deidentification')

In [2]:
from model import DocumentProfileMatchingTransformer

checkpoint_path = "/home/jxm3/research/deidentification/unsupervised-deidentification/saves/deid-wikibio_deid_exp/okpvvffw_46/checkpoints/epoch=7-step=1823.ckpt"
model = DocumentProfileMatchingTransformer.load_from_checkpoint(
    checkpoint_path,
    dataset_name='wiki_bio',
    model_name_or_path='distilbert-base-uncased',
    num_workers=1,
    loss_fn='exact',
    num_neighbors=2048,
    base_folder="/home/jxm3/research/deidentification/unsupervised-deidentification",
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialized DocumentProfileMatchingTransformer with learning_rate = 0.0002


In [3]:
from datamodule import WikipediaDataModule
import os

num_cpus = os.cpu_count()

dm = WikipediaDataModule(
    model_name_or_path='distilbert-base-uncased',
    dataset_name='wiki_bio',
    num_workers=min(8, num_cpus),
    train_batch_size=64,
    eval_batch_size=64,
    max_seq_length=64,
    redaction_strategy="",
    base_folder="/home/jxm3/research/deidentification/unsupervised-deidentification",
)
dm.setup("fit")

Initializing WikipediaDataModule with num_workers = 8


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)
Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-5535f82839d9fec4.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-5b1c3941089b7f1b.arrow


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

## Get the birthday data

In [66]:
from typing import Tuple

from collections import OrderedDict

import datasets

class WikiDataset:
    dataset: datasets.Dataset
    
    def __init__(self, dm: WikipediaDataModule):
        self.shuffled = True
        self.train_dataset = dm.train_dataset
        self.val_dataset = dm.val_dataset
        self.train_label_names = list(dm.train_dataset['name'])
        self.val_label_names = list(dm.val_dataset['name'])
        

In [67]:
from textattack import Attack
from textattack.constraints.pre_transformation import RepeatModification

#model_wrapper = MyModelWrapper(model, dm.tokenizer)
#model_wrapper.to('cuda')
wd = WikiDataset(dm)

In [69]:
dataset.train_dataset[320]['profile'].split(r'|\n')

['eye_color | blue\nname | anja Šaranović\nbirth_name | anja Šaranović\ncompetitions | miss miss international 2010 -lrb- top 15 -rrb- miss universe 2011 kraljevo -lrb- winner -rrb- miss serbia universe 2010\nimage | anja.jpg\nbirth_place | kraljevo , yugoslavia\nhair_color | blonde\nbirth_date | 12 september 1989\narticle_title | anja Šaranović']

In [72]:
from typing import List, Tuple

def process_dataset(_dataset) -> List[Tuple[int, int]]:
    _processed_data = []
    for idx, d in enumerate(_dataset):
        profile = d['profile']
        date_str_matches = re.search(r"birth_date \| ([\d]{1,4} [a-z]+ [\d]{1,4})", profile)
        if date_str_matches:
            date_str = date_str_matches.group(1)
            # print(date_str)
            # parse to datetime.datetime
            try:
                dt = datetime.datetime.strptime(date_str, "%d %B %Y")
            except ValueError as e:
                # print(e)
                continue
            day_class_num = (dt.month - 1) * 31 + dt.day
            _processed_data.append([idx, day_class_num])
    return _processed_data


train_birthday_data = process_dataset(wd.train_dataset)
val_birthday_data = process_dataset(wd.val_dataset)

print('dataset lengths:', len(train_birthday_data), '/', len(val_birthday_data))

dataset lengths: 43032 / 10761


In [63]:
import datetime

d = datetime.datetime.strptime('17 january 1943', "%d %B %Y")
d.day

17

## Create the birthday model

In [73]:
import torch
import transformers

from pytorch_lightning import LightningModule

class BirthdayModel(LightningModule):
#     TODO impl birthday model below
    profile_embeddings: torch.Tensor
    classifier: torch.nn.Module
    max_seq_length: int
    
    def __init__(self, model: DocumentProfileMatchingTransformer, tokenizer: transformers.PreTrainedTokenizer, max_seq_length: int = 64):
        self.train_profile_embeddings = torch.tensor(model.train_embeddings)
        self.val_profile_embeddings = torch.tensor(model.val_embeddings)
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=0.1),
            # 12 * 31 possible outputs
            torch.nn.Linear(model.val_embeddings.shape[1], 12*31),
        )
                 
    def to(self, device):
        self.train_profile_embeddings.to(device)
        self.val_profile_embeddings.to(device)
        self.classifier.to(device)
        return self # so semantics `model = MyModelWrapper().to('cuda')` works properly

    def call_train(self, train_indices: torch.Tensor):
        prof_embeddings = self.train_profile_embeddings[train_indices]
        return self.classifier(prof_embeddings)
        
    def call_val(self, val_indices: torch.Tensor):
        prof_embeddings = self.val_profile_embeddings[val_indices]
        return self.classifier(prof_embeddings)

    def training_step(??):
    
    def validation_step(??):
            